In [6]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img
from tensorflow.keras.utils import img_to_array
from sklearn.metrics import classification_report, accuracy_score
import os
import glob

## Data Augentation

In [4]:
# # Define data augmentation parameters (adjust as needed)
# datagen = ImageDataGenerator(
#     rotation_range=50,  # Rotate images by up to 40 degrees
#     shear_range=0.2,     # Shear by up to 20%
#     zoom_range=0.2,      # Zoom by up to 20%
#     horizontal_flip=True,# Randomly flip images horizontally
#     vertical_flip = True,
#     brightness_range=(0.5, 1.5)  # Adjust brightness range
# )

# # Define path to your image folder
# image_dir = "Train/silver"

# # Define output directory for augmented images (create if it doesn't exist)
# save_to_dir = "Train/silver"
# import os
# os.makedirs(save_to_dir, exist_ok=True)  # Create directory if it doesn't exist

# # Loop through each image in the folder
# for filename in os.listdir(image_dir):
#     # Load the image
#     img = load_img(os.path.join(image_dir, filename))

#     # Convert to array
#     x = img_to_array(img)

#     # Reshape (optional, based on your model's requirements)
#     x = x.reshape((1,) + x.shape)  # Reshape if your model expects a specific format

#     # Generate and save augmented images
#     i = 0
#     for batch in datagen.flow(x, batch_size=1, save_to_dir=save_to_dir, save_prefix="aug_", save_format="jpeg"):
#         i += 1
#         # You can adjust the number of augmented images per original image here (e.g., i > 5)
#         if i > 1:
#             break
# print(f"Successfully augmented and saved images to {save_to_dir}")


Successfully augmented and saved images to Train/silver


## Modeling

In [5]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False)

In [6]:
x = tf.keras.layers.GlobalAvgPool2D()(base_model.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(3,activation="softmax")(x)

model = tf.keras.Model(inputs = base_model.input , outputs = x)


In [7]:
#freezing the original model's layers
for layer in base_model.layers:
    layer.trainable = False

In [8]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [9]:
# define the directory containing training images
target_size = (224, 224)

# initialize lists to store images and labels
images = []
labels = []

for filename in os.listdir("Train/Material/gold"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Train/gold", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(0)
for filename in os.listdir("Train/Material/silver"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Train/silver", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(1)
for filename in os.listdir("Train/Material/bronze"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Train/bronze", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(2)
        
# convert lists to numpy arrays
X_train = np.array(images)
Y_train = np.array(labels)

c:\Users\ghass\anaconda3\envs\yolo\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [12]:
X_train.shape

(1216, 224, 224, 3)

In [13]:
model.fit(X_train,Y_train,epochs = 10)

Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 21s 413ms/step - accuracy: 0.5960 - loss: 0.8912
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 395ms/step - accuracy: 0.8695 - loss: 0.3280
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 408ms/step - accuracy: 0.9453 - loss: 0.1673
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 387ms/step - accuracy: 0.9504 - loss: 0.1265
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 15s 385ms/step - accuracy: 0.9688 - loss: 0.0900
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 22s 583ms/step - accuracy: 0.9793 - loss: 0.0509
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 23s 611ms/step - accuracy: 0.9853 - loss: 0.0485
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 14s 375ms/step - accuracy: 0.9945 - loss: 0.0279
Epoch 9/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 352ms/step - accuracy: 0.9947 - loss: 0.0226
Epoch 10/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 345ms/step - accuracy: 0.9968 - loss: 0.0166


In [14]:
# define the directory containing training images
target_size = (224, 224)

# initialize lists to store images and labels
images = []
labels = []

for filename in os.listdir("Test/Material/gold"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Test/gold", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(0)
for filename in os.listdir("Test/Material/silver"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Test/silver", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(1)
for filename in os.listdir("Test/Material/bronze"):
    if filename.endswith('.jpg') or filename.endswith("png") or filename.endswith("jpeg"):
        # load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(os.path.join("Test/bronze", filename), target_size=target_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
       # add the preprocessed image to the list of images
        images.append(img_array)
        labels.append(2)
        
# convert lists to numpy arrays
X_test = np.array(images)
Y_test = np.array(labels)

In [28]:
test_Path = "Test/Material"
IMAGE_SIZE = (224,224)
folders = glob.glob(train_Path + '/*')
folders

['Test/Material\\bronze', 'Test/Material\\gold', 'Test/Material\\silver']

In [29]:
labels = ["bronze","gold","silver"]

In [30]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_Path = "Test/Material"
test_set = test_datagen.flow_from_directory(
    test_Path,  # Use your test data directory here
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle= False
)

Found 76 images belonging to 3 classes.


In [31]:
model = tf.keras.models.load_model("MaterialModelV2.keras")

c:\Users\ghass\anaconda3\envs\yolo\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 326 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [37]:
test_labels = test_set.classes
# Change the list because the order of the training is different from the order of testing
test_labels1 = [2 if x == 0 else 0 if x == 1 else 1 for x in test_labels]
# Predict the labels for the test set
predictions = model.predict(test_set, steps=len(test_set))
predicted_classes = np.argmax(predictions, axis=1)

# Get the class indices to labels mapping
class_labels = list(test_set.class_indices.keys())

# Print classification report
report = classification_report(test_labels1, predicted_classes, target_names=class_labels)
print(report)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 302ms/step
              precision    recall  f1-score   support

      bronze       1.00      0.95      0.97        57
        gold       0.87      1.00      0.93        13
      silver       0.86      1.00      0.92         6

    accuracy                           0.96        76
   macro avg       0.91      0.98      0.94        76
weighted avg       0.97      0.96      0.96        76

